In [1]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available
print(torch.cuda.device_count())  # Should be > 0 if GPU is accessible
print(torch.cuda.get_device_name(0))  # Displays the GPU model


True
1
NVIDIA GeForce RTX 2080 SUPER


In [31]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import math
import random
import shutil
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate,ZeroPadding3D

In [ ]:
video_path='/content/Echonet/EchoNet-Dynamic/Videos'
all_videos=os.listdir(video_path)
print(len(all_videos))

10031


In [ ]:
#randomly select 50 videos
selected_videos=random.sample(all_videos,50)
print(len(selected_videos))
dest_path='/content/drive/MyDrive/Selected_Videos'
os.makedirs(dest_path,exist_ok=True)
for video in selected_videos:
    shutil.copy(os.path.join(video_path,video),os.path.join(dest_path,video))

50


In [ ]:
# extract frames from video and visualize these frames
new_video_path='/content/Selected_Videos'
def extract_frames(videopath):
    cap=cv2.VideoCapture(videopath)
    frames=[]
    if not cap.isOpened():
        print(f'error opening {videopath} ')
    while cap.isOpened():
        ret,frame=cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

frames_dict={}


for video in os.listdir(new_video_path):
    frames=extract_frames(os.path.join(new_video_path,video))
    frames_dict[video]=frames



In [3]:
# extract frames from video and visualize these frames
new_video_path='Selected_Videos'
def extract_frames(videopath,sampling_rate=2):
    cap=cv2.VideoCapture(videopath)
    frames=[]
    frame_count=0
    if not cap.isOpened():
        print(f'error opening {videopath} ')
    while cap.isOpened():
        ret,frame=cap.read()
        if not ret:
            break
        frame_rgb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

        if frame_count % sampling_rate == 0:
            frames.append(frame_rgb)
        frame_count+=1


        # gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        # gray=np.expand_dims(gray,axis=-1)
        # print(gray.shape)
    cap.release()
    return frames

frames_dict={}


for video in os.listdir(new_video_path):
    frames=extract_frames(os.path.join(new_video_path,video))
    frames_dict[video]=frames

print(len(frames_dict))
#


50


In [5]:
# split the data
file_list=pd.read_csv('EchoNet-Dynamic/EchoNet-Dynamic/FileList.csv')
file_list.head()

,FileName,EF,ESV,EDV,FrameHeight,FrameWidth,FPS,NumberOfFrames,Split
0,0X100009310A3BD7FC,78.498406,14.881368,69.210534,112,112,50,174,VAL
1,0X1002E8FBACD08477,59.101988,40.383876,98.742884,112,112,50,215,TRAIN
2,0X1005D03EED19C65B,62.363798,14.267784,37.909734,112,112,50,104,TRAIN
3,0X10075961BC11C88E,54.545097,33.143084,72.914210,112,112,55,122,TRAIN
4,0X10094BA0A028EAC3,24.887742,127.581945,169.855024,112,112,52,207,VAL


In [6]:
train_files=file_list[file_list['Split']=='TRAIN']['FileName'].tolist()
val_files=file_list[file_list['Split']=='VAL']['FileName'].tolist()
test_files=file_list[file_list['Split']=='TEST']['FileName'].tolist()


In [7]:
len(train_files),len(val_files),len(test_files)

(7465, 1288, 1277)

In [8]:
train_frames=[frames_dict[f'{file}.avi'] for file in train_files if f'{file}.avi' in frames_dict]
val_frames=[frames_dict[f'{file}.avi'] for file in val_files if f'{file}.avi' in frames_dict]
test_frames=[frames_dict[f'{file}.avi'] for file in test_files if f'{file}.avi' in frames_dict]

In [9]:
len(train_frames),len(val_frames),len(test_frames)

(35, 7, 8)

In [10]:
#create segmentation mask
volume_tracings=pd.read_csv('Echonet-Dynamic/EchoNet-Dynamic/VolumeTracings.csv')

In [11]:
def create_mask(image_shape,contour_points):
    mask=np.zeros(image_shape,dtype=np.uint8)
    contour=np.array(contour_points,dtype=np.int32).reshape(-1,1,2)
    cv2.fillPoly(mask,[contour],1)
    return mask

In [12]:
def generate_masks(frames, file_name):
    masks = []
    for frame_number, frame in enumerate(frames):
        # Check if the file and frame exist in volume_tracings
        matching_rows = volume_tracings[(volume_tracings['FileName'] == file_name) & (volume_tracings['Frame'] == frame_number)]
        if matching_rows.empty:
            # Handle the case where there are no matching rows, e.g., skip the frame or create an empty mask
            # Here, we create an empty mask
            mask = np.zeros(frame.shape, dtype=np.uint8)
        else:
            contour_points = [(matching_rows[f'X{i}'].values[0],
                               matching_rows[f'Y{i}'].values[0])
                              for i in range(1, (len(volume_tracings.columns) - 2) // 2 + 1)]
            mask = create_mask(frame.shape, contour_points)
        masks.append(mask)
    return masks

In [15]:
len(train_frames),len(frames_dict)
#train_masks = [generate_masks(frames, file) for frames, file in zip(train_frames, train_files)]

(35, 50)

In [14]:
train_masks = [generate_masks(frames, file) for frames, file in zip(train_frames, train_files)]
val_masks = [generate_masks(frames, file) for frames, file in zip(val_frames, val_files)]
test_masks = [generate_masks(frames, file) for frames, file in zip(test_frames, test_files)]

In [16]:
import pickle

# Save train_masks
with open('train_masks.pkl', 'wb') as f:
    pickle.dump(train_masks, f)

# Save val_masks
with open('val_masks.pkl', 'wb') as f:
    pickle.dump(val_masks, f)

# Save test_masks
with open('test_masks.pkl', 'wb') as f:
    pickle.dump(test_masks, f)

In [17]:
#preprocess it

def normalize_frames(frames):
    nf=[frame/255.0 for frame in frames]
    return nf

def resize(frames,size=(112,112)):
    nr=[cv2.resize(frame,size) for frame in frames]
    return nr

def pad_or_truncate(frames, max_frames=250):
    if len(frames) > max_frames:
        return frames[:max_frames]
    else:
        return frames + [frames[-1]] * (max_frames - len(frames))

def preprocess(frames,masks):
    rf=resize(frames)
    nf=normalize_frames(rf)
    pt=pad_or_truncate(nf)

    rm=resize(masks)
    nm=normalize_frames(rm)
    ptm=pad_or_truncate(nm)

    return pt ,ptm



In [18]:
preprocessed_train = [preprocess(frames, masks) for frames, masks in zip(train_frames, train_masks)]
preprocessed_val = [preprocess(frames, masks) for frames, masks in zip(val_frames, val_masks)]
preprocessed_test = [preprocess(frames, masks) for frames, masks in zip(test_frames, test_masks)]

In [19]:
prerocessed_train_frames, preprocessed_train_masks = zip(*preprocessed_train)
prerocessed_val_frames, preprocessed_val_masks = zip(*preprocessed_val)
prerocessed_test_frames, preprocessed_test_masks = zip(*preprocessed_test)

In [ ]:

def unet_model(input_size=(250,112, 112, 3)):
    inputs = Input(input_size)
    conv1 = Conv3D(64,(3,3,3) , activation='relu', padding='same')(inputs)
    conv1 = Conv3D(64, (3,3,3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling3D(pool_size=(2,2,2))(conv1)

    conv2 = Conv3D(128, (3,3,3), activation='relu', padding='same')(pool1)
    conv2 = Conv3D(128, (3,3,3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling3D(pool_size=(2,2,2))(conv2)

    conv3 = Conv3D(256,(3,3,3), activation='relu', padding='same')(pool2)
    conv3 = Conv3D(256, (3,3,3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling3D(pool_size=(2,2,2))(conv3)

    conv4 = Conv3D(512, (3,3,3), activation='relu', padding='same')(pool3)
    conv4 = Conv3D(512, (3,3,3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling3D(pool_size=(2,2,2))(conv4)

    conv5 = Conv3D(1024, (3,3,3), activation='relu', padding='same')(pool4)
    conv5 = Conv3D(1024, (3,3,3), activation='relu', padding='same')(conv5)

    up6 = UpSampling3D(size=(2,2,2))(conv5)
    pad_up6 = ZeroPadding3D(((0, 1), (0, 0), (0, 0)))(up6)
    merge6 = concatenate([conv4, pad_up6], axis=4)
    conv6 = Conv3D(512,(3,3,3), activation='relu', padding='same')(merge6)
    conv6 = Conv3D(512, (3,3,3), activation='relu', padding='same')(conv6)

    up7 = UpSampling3D(size=(2,2,2))(conv6)
    merge7 = concatenate([conv3, up7], axis=4)
    conv7 = Conv3D(256, (3,3,3), activation='relu', padding='same')(merge7)
    conv7 = Conv3D(256, (3,3,3), activation='relu', padding='same')(conv7)

    up8 = UpSampling3D(size=(2,2, 2))(conv7)
    pad_up8 = ZeroPadding3D(((0, 1), (0, 0), (0, 0)))(up8)
    merge8 = concatenate([conv2, pad_up8], axis=4)
    conv8 = Conv3D(128, (3,3,3), activation='relu', padding='same')(merge8)
    conv8 = Conv3D(128, (3,3,3), activation='relu', padding='same')(conv8)

    up9 = UpSampling3D(size=(2,2,2))(conv8)
    merge9 = concatenate([conv1, up9], axis=4)
    conv9 = Conv3D(64, (3,3,3), activation='relu', padding='same')(merge9)
    conv9 = Conv3D(64, (3,3,3), activation='relu', padding='same')(conv9)

    conv10 = Conv3D(1, (1,1,1), activation='sigmoid')(conv9)

    model = Model(inputs, conv10)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

model = unet_model()

In [35]:
X_train, y_train = zip(*preprocessed_train)
X_train = np.array(X_train)
y_train = np.array(y_train)

In [37]:
y_train.shape

(35, 250, 112, 112, 3)

In [38]:
y_train = np.mean(y_train, axis=-1, keepdims=True)
y_train.shape

(35, 250, 112, 112, 1)

In [39]:
X_train.shape

(35, 250, 112, 112, 3)

In [41]:
from tensorflow.keras.backend import clear_session

clear_session()  # Clear any existing TensorFlow graph
model = unet_model()  # Redefine the model
model.fit(X_train, y_train, epochs=5, batch_size=8, validation_split=0.2)


Epoch 1/5


ResourceExhaustedError: Graph execution error:

Detected at node functional_1/concatenate_3_1/concat defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\atabass4\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\atabass4\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\atabass4\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\atabass4\AppData\Roaming\Python\Python312\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\atabass4\Lib\asyncio\base_events.py", line 641, in run_forever

  File "c:\Users\atabass4\Lib\asyncio\base_events.py", line 1987, in _run_once

  File "c:\Users\atabass4\Lib\asyncio\events.py", line 88, in _run

  File "C:\Users\atabass4\App

In [ ]:
X_val, y_val = zip(*preprocessed_val)
X_val = np.array(X_val)
y_val = np.array(y_val)

X_test, y_test = zip(*preprocessed_test)
X_test = np.array(X_test)
y_test = np.array(y_test)

# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
import matplotlib.pyplot as plt

# Visualize a sample prediction
sample_index = 0
predicted_mask = model.predict(np.expand_dims(X_test[sample_index], axis=0))[0]

plt.figure(figsize=(10, 5))
plt.subplot(1, 3, 1)
plt.title("Original Frame")
plt.imshow(X_test[sample_index], cmap='gray')

plt.subplot(1, 3, 2)
plt.title("Ground Truth Mask")
plt.imshow(y_test[sample_index], cmap='gray')

plt.subplot(1, 3, 3)
plt.title("Predicted Mask")
plt.imshow(predicted_mask, cmap='gray')

plt.show()

In [ ]:
#loop to iterate through folder to check frame size for all videos and fps
for v in os.listdir(video_path):
    if v.endswith('.avi'):
        path=os.path.join(video_path,v)
        w,h,c,fps=get_frame_size(path)
        print(f'frame size is {w}x{h} and count is {c} with fps equals {fps}')


In [ ]:
# extract frames from video and visualize these frames
def extract_frames(videopath):
    cap=cv2.VideoCapture(videopath)
    frames=[]
    if not cap.isOpened():
        print('error')
    while True:
        ret,frame=cap.read()
        if not ret:
            break
        #frame=cv2.resize(frame,(112,112))
        frames.append(frame)
    cap.release()
    return frames








In [ ]:
video_path='/content/Echonet/EchoNet-Dynamic/Videos'
frames_dict={}
for v in os.listdir(video_path):
  path=os.path.join(video_path,v)
  frames=extract_frames(path)
  frames_dict[v]=frames

In [ ]:
# function to visualize all frames
def visualizeframes(frames,framesperrow):
    num_frames=len(frames)
    num_rows = math.ceil(num_frames /framesperrow)
    fig, axes = plt.subplots(num_rows, framesperrow, figsize=(20, num_rows * 4))
    axes = axes.flatten()
    for i,frame in enumerate(frames):
        axes[i].imshow(frame)
        axes[i].axis('off')
    plt.show()


# visualizeframes(f,5)
#print('total number of frames in this video is :', len(f))

In [ ]:
# to play multiple videos in a folder
flag=0
def play_video(video_path):
    global flag
    cap=cv2.VideoCapture(video_path)
    cv2.namedWindow('Video',cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Video',800,600)
    while cap.isOpened():
        ret,frame=cap.read()
        if not ret:
            break
        cv2.imshow('Video',frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            flag=1
            break
    cap.release()
    cv2.destroyAllWindows()





In [ ]:
import pdb

In [ ]:
#loop to iterate through folder
for v in os.listdir(video_path):
    if v.endswith('.avi'):
        path=os.path.join(video_path,v)
        if flag==0:
            print(f'playing {v}')
            play_video(path)
        else:
            print('interrupted')
            break


In [ ]:
#loop to iterate through folder
# this block of code iterates through all vides, extracts frames, normalizes those frames and performs padding or truncation to make sure all videos have same number of frames
# These frames are stored in list and then converted to an array to be used as input for model.
os.makedirs('training',exist_ok=True)
os.makedirs('testing',exist_ok=True)
os.makedirs('validation',exist_ok=True)
df=pd.read_csv('/content/Echonet/EchoNet-Dynamic/FileList.csv')
pad_trunc=[]
for v in os.listdir(video_path):
    if v.endswith('.avi'):
        path=os.path.join(video_path,v)
        frames=extract_frames(path)
        normalized=normalize_frames(frames)
        padtruncframes=pad_or_truncate(normalized)
        print(f'number of frames in {path} is {len(padtruncframes)}')
    pad_trunc.append(padtruncframes)
    print(len(pad_trunc))

    # split files into training, testing and validation
    file_name=os.path.splitext(v)[0]
    split_type=df.loc[df['FileName']==file_name,'Split'].values[0]

    if split_type=='TRAIN':
        shutil.move(f'{path}',f'training/{file_name}.avi')
    elif split_type == 'TEST':
        shutil.move(f'{path}',f'testing/{file_name}.avi')
    elif split_type == 'VAL':
        shutil.move(f'{path}',f'validation/{file_name}.avi')

    print('files have been successfully split')

        #w,h,c,fps=get_frame_size(path)
        #print(f'frame size is {w}x{h} and count is {c} with fps equals {fps}')
# saving list of processed frames in numpy array to use in model
padtruncarray=np.array(pad_trunc)

In [ ]:
import pandas as pd


In [ ]:
volume_tracings=pd.read_csv('/content/Echonet/EchoNet-Dynamic/VolumeTracings.csv')